# Fit energies

to do:

0. Rename FandeDataModuleASE into FandeDataModule
1. Improve FandeDataModuleASE, initialization with forces and energies
2. make parallel calculation of invariants within

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.expandvars("/home/$USER/repos/fande/") )

In [ ]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
traj_355 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")

traj_295_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
traj_355_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
traj_295_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
traj_355_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")



In [ ]:
# traj_train = traj_295[0:5000:10] + traj_355[0:5000:10] + traj_295_2000K[0:5000:10] + traj_355_2000K[0:5000:10] + traj_295_2000K_forced[0:5000:10] + traj_355_2000K_forced[0:5000:10]
traj_train = traj_295_2000K[1000:5000:5] + traj_355_2000K[1000:5000:5]  
traj_train = traj_train.copy()

In [ ]:
energies_train = [s.get_potential_energy() for s in traj_train]

In [ ]:
# just to fake the initialization of the fdm
# Hyperparameters:
hparams = {
        'dtype' : 'float32',
        'device' : 'gpu'
        }

from fande.data import FandeDataModuleASE

## Train data:
energies_train = np.zeros(len(traj_train) )
forces_train = np.zeros( (len(traj_train), len(traj_train[0]), 3 ) )

for i, snap in enumerate(traj_train):
        energies_train[i] = snap.get_potential_energy()
        forces_train[i] = snap.get_forces()
train_data = {'trajectory': traj_train, 'energies': energies_train,'trajectory_energies': traj_train, 'forces': forces_train}

## Test data:
# energies_test = np.zeros(len(traj_test) )
# forces_test = np.zeros( (len(traj_test), len(traj_test[0]), 3 ) )
# for i, snap in enumerate(traj_test):
#         energies_test[i] = snap.get_potential_energy()
#         forces_test[i] = snap.get_forces()
# test_data = {'trajectory_energies': traj_test, 'energies': energies_test, 'forces': forces_test}

test_data={}


fdm = FandeDataModuleASE(train_data, train_data, hparams)

In [694]:
# snap_copy = traj_train[0].copy()
snap_copy = io.read("/data1/simulations/datasets/rotors/simulations_cluster/27_11_2023/27/2023-11-27_17:27:26.502178/md_trajectory.traj", index="0")

from ase.build import make_supercell

snap_super = make_supercell(snap_copy, [[1,0,0],[0,1,0],[0,0,1]])

In [690]:
from ase.visualize import view

view(snap_super)

<Popen: returncode: None args: ['/home/dlbox2/anaconda3/envs/fande/bin/pytho...>

In [695]:
len(snap_super)

684

In [725]:
%%time
import numpy as np

# def calculate_invariants_librascal_no_derivatives(
#         self,
#         trajectory, 
#         soap_params,
#         frames_per_batch=10): 

# Descriptors parameters:
# https://github.com/lab-cosmo/librascal/blob/master/examples/MLIP_example.ipynb
# soap_params = {
# # 'species': ["H", "C", "O", "N", "Si"],
# 'periodic': True,
# 'interaction_cutoff': 3.0,
# 'gaussian_sigma_constant': 0.3,
# 'max_radial': 5,
# 'max_angular': 5,
# 'cutoff_smooth_width': 0.1,
# # 'average': "off",
# # 'crossover': True,
# # 'dtype': "float64",
# # 'n_jobs': 10,
# # 'sparse': False,
# # 'positions': [7, 11, 15] # ignored
# }


# train_X_np = fdm.calculate_invariants_librascal_no_derivatives(traj_train[0:10], soap_params, frames_per_batch=1)
# X = fdm.calculate_invariants_librascal_no_derivatives([snap_super], soap_params, frames_per_batch=1)
hypers = dict(soap_type="PowerSpectrum",
        interaction_cutoff=3.0,
        max_radial=5,
        max_angular=5,
        gaussian_sigma_constant=0.3,
        gaussian_sigma_type="Constant",
        cutoff_function_type="RadialScaling",
        cutoff_smooth_width=0.1, # 0.1 is way better than 0.5
        cutoff_function_parameters=
                dict(
                        rate=1,
                        scale=3.5,
                        exponent=4
                        ),
        radial_basis="GTO",
        normalize=True, # setting False makes model untrainable
        #   optimization=
        #         dict(
        #                 Spline=dict(
        #                    accuracy=1.0e-05
        #                 )
        #             ),
        compute_gradients=True,
        expansion_by_species_method='structure wise'
        )
        
# snap_super.set_pbc([False, False, False])
snap_super.set_pbc([True, True, True])
# snap_super

fdm.soap_hypers = hypers
fdm.atomic_groups_train = [list(range(len(snap_super)))]
fdm.centers_positions_train = [list(range(len(snap_super)))]
fdm.derivatives_positions_train = [list(range(len(snap_super)))]
X = fdm.calculate_invariants_librascal_no_derivatives([snap_super], soap_params, frames_per_batch=1)
# DX = fdm.calculate_snapshot_invariants_librascal(snap_super, same_centers_derivatives=True)

Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.11it/s]


CPU times: user 1.96 s, sys: 200 ms, total: 2.16 s
Wall time: 1.16 s


In [ ]:
indices_non_H = []

for ind,s in enumerate(traj_train[0].get_chemical_symbols()):
        # print(s)
        if s != "H":
                indices_non_H.append(ind)

energies_train_normalized = (energies_train - np.min(energies_train))/(np.max(energies_train) - np.min(energies_train))

# train_X = X_np[:,indices_non_H,:]
train_X = train_X_np

# train_X = torch.tensor(X_np_batched)
train_X = torch.tensor(train_X.sum(axis=1)).cuda()
train_Y = torch.tensor(energies_train_normalized).cuda()

print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

In [ ]:
import torch
import gpytorch
import numpy as np

from gpytorch.kernels import (
    RBFKernel,
    ScaleKernel,
    LinearKernel,
    AdditiveKernel,
    MultitaskKernel,
    PolynomialKernel,
    MaternKernel,
)
from gpytorch.means import ZeroMean, ConstantMean

from gpytorch.models import ExactGP

import torch, torch.nn as nn


# https://stackoverflow.com/questions/76251549/setting-the-task-covariance-matrix-to-the-correlation-matrix-in-gpytorch

class ExactGPModelEnergies(ExactGP):
    def __init__(self, train_X, train_Y, likelihood):
        super().__init__(
            train_X, train_Y, likelihood
        )  # the old-style super(ExactGPModel, self) was causing error!
        self.mean_module = ConstantMean()

        self.soap_dim = train_X.shape[-1]
        # self.covar_module = ScaleKernel(RBFKernel(ard_num_dims=self.soap_dim))#LinearKernel()
        self.covar_module = ScaleKernel(MaternKernel(ard_num_dims=self.soap_dim))#LinearKernel()
        # self.covar_module = ScaleKernel(LinearKernel(ard_num_dims=self.soap_dim))

    def forward(self, X):
        x = X
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)



In [ ]:
# train_X = train_X.cuda()
# train_Y = torch.tensor(energies_train_normalized).cuda()
# test_X = test_X.cuda()
# test_Y = test_Y.cuda()
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModelEnergies(train_X, train_Y, likelihood)
# model = ExactGPModelEnergiesMulti(train_X, train_Y, likelihood)
model = model.cuda()
likelihood = likelihood.cuda()

In [ ]:
train_Y.shape

In [ ]:
# this is for running the notebook in our testing framework
import os



training_iter = 100


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

print("Training...")
# initial_loss = -mll(model(train_X), train_Y).item()
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # print("grad zeroed")
    # Output from model
    output = model(train_X)
    # print("Output calculated")
    # print(output)
    # Calc loss and backprop gradients
    loss = -mll(output, train_Y)
    # print("Loss calculated, ")

    loss.backward()
    # print(loss)
    # print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
    #     i + 1, training_iter, loss.item(),
    #     model.covar_module.base_kernel.lengthscale.item(),
    #     model.likelihood.noise.item()
    # ))
    print(f"step: {i},loss: {loss}")
    optimizer.step()

# print("Initial loss: ", initial_loss)
print("Final loss: ", loss)


# edit     def calculate_invariants_librascal_no_derivatives(

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

test_x = test_X

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
# with torch.no_grad(), gpytorch.settings.fast_pred_var():
#     test_x = torch.linspace(0, 1, 51)
observed_pred = likelihood(model(test_x)).mean.detach().cpu().numpy()



import matplotlib.pyplot as plt

plt.plot(observed_pred, label="Predicted energy")
plt.plot(test_Y.detach().cpu().numpy(), label="True energy")
plt.xlabel('step')
plt.ylabel('Predicted Energy')
# plt.xlim(0, 100)
plt.legend()
plt.show()

In [ ]:
plt.plot(test_Y)
plt.ylabel('Energy')
plt.xlabel('step')
plt.xlim(85, 100)
plt.show()

In [ ]:
traj_test = io.read("/data1/simulations/datasets/rotors/for jacs paper/295_0.075_same_+/md_trajectory.traj", index=":", format="traj")
energies_test = [s.get_potential_energy() for s in traj_test]
energies_test_normalized = (energies_test - np.min(energies_train))/(np.max(energies_train) - np.min(energies_train))
traj = traj_test


In [ ]:
test_X_np = fdm.calculate_invariants_librascal_no_derivatives(traj_test[0:10], soap_params, frames_per_batch=1)

In [ ]:
indices_non_H = []

for ind,s in enumerate(traj[0].get_chemical_symbols()):
        # print(s)
        if s != "H":
                indices_non_H.append(ind)

import numpy as np

X_np_batched = np.array(X_np_batched)
E_np_batched = np.array(E_np_batched).flatten()

# test_X = X_np_batched[:,indices_non_H,:]
test_X = X_np_batched
test_X = test_X.sum(axis=1)
test_X = torch.tensor(test_X).cuda()
test_Y = torch.tensor(energies_test_normalized).cuda()

In [ ]:
from matplotlib import pyplot as plt

plt.plot(energies_train_normalized)
# plt.plot(energies_test_normalized)
plt.plot(test_Y.detach().cpu().numpy())
plt.show()

# Drafting the energy model

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.expandvars("/home/$USER/repos/fande/") )

In [1]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
traj_355 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")

traj_295_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
traj_355_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
traj_295_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
traj_355_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")


In [4]:
from fande.data import FandeDataModule

fdm = FandeDataModule()

fande.models module imported...


In [5]:
soap_params = dict(soap_type="PowerSpectrum",
        interaction_cutoff=1.0,
        max_radial=1,
        max_angular=1,
        gaussian_sigma_constant=0.3,
        gaussian_sigma_type="Constant",
        cutoff_function_type="RadialScaling",
        cutoff_smooth_width=0.1, # 0.1 is way better than 0.5
        cutoff_function_parameters=
                dict(
                        rate=1,
                        scale=3.5,
                        exponent=4
                        ),
        radial_basis="GTO",
        normalize=True, # setting False makes model untrainable
        #   optimization=
        #         dict(
        #                 Spline=dict(
        #                    accuracy=1.0e-05
        #                 )
        #             ),
        compute_gradients=True,
        expansion_by_species_method='structure wise'
        )

rate = 100
# traj_train = traj_295[0:5000:10] + traj_355[0:5000:10] + traj_295_2000K[0:5000:10] + traj_355_2000K[0:5000:10] + traj_295_2000K_forced[0:5000:10] + traj_355_2000K_forced[0:5000:10]
traj_train = traj_295[0:5000:rate] + traj_355[0:5000:rate] + traj_295_2000K[0:5000:rate] + traj_355_2000K[0:5000:rate] + traj_295_2000K_forced[0:5000:rate] + traj_355_2000K_forced[0:5000:rate]

train_energies = [s.calc.get_potential_energy() for s in traj_train]
emax = max(train_energies)
emin = min(train_energies)
train_energies = [(e - emin)/(emax - emin) for e in train_energies]

fdm.soap_hypers = soap_params
fdm.atomic_groups_train = [list(range(len(traj_train[0])))]
train_data_loader = fdm.prepare_train_data_loaders_energy(traj_train, train_energies, soap_params)

Total length of traj is 300
Total number of batches 300
Calculating invariants on trajectory with librascal...


100%|██████████| 300/300 [00:11<00:00, 26.17it/s]
/home/dlbox2/repos/fande/fande/data/data_module.py:186: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/torch/csrc/utils/tensor_new.cpp:245.)
  X = torch.tensor(X_np_batched,dtype=torch.float32)


In [6]:
from fande.models import EnergyModel, RawEnergyModel

#     def __init__(
#             self,
#             energy_model,
#             energy_train_data_loaders: list,
#             fdm=None, # specification of fdm is optional
#             hparams=None,
#             gpu_id=None

hparams = {
        'dtype' : 'float32',
        'device' : 'gpu',
        'energy_model_hparams' : {
                'num_epochs' : 100,
                'learning_rate' : 0.01,
        }
        }


raw_energy_model = RawEnergyModel(
        train_x = train_data_loader.dataset[:][0],
        train_y = train_data_loader.dataset[:][1],
        hparams = hparams)

        
Energy_model = EnergyModel(
        raw_energy_model,
        train_data_loader,
        hparams=hparams, 
        gpu_id=0)



/home/dlbox2/anaconda3/envs/fande/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dlbox2/anaconda3/envs/fande/lib/python3.10/sit ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



RawEnergyModel initialized


/home/dlbox2/anaconda3/envs/fande/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
/home/dlbox2/anaconda3/envs/fande/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'energy_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['energy_model'])`.
  rank_zero_warn(


In [7]:
Energy_model.fit()

Training energy model


/home/dlbox2/anaconda3/envs/fande/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/dlbox2/anaconda3/envs/fande/lib/python3.10/sit ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name       | Type                       | Params
----------------------------------------------------------
0 | likelihood | GaussianLikelihood         | 1     
1 | model      | ExactGPModelEnergy         | 32    
2 | mll        | ExactMarginalLogLikelihood | 32    
----------------------------------------------------------
32        Trainable params
0         Non-trainable params
32        Total params
0.000     Total estimated model params size (MB)
/home/dlbox2/anaconda3/envs/fande/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [8]:
from fande.predict import FandePredictor
from fande.ase import FandeCalc

AG_force_model = None
predictor = FandePredictor(
        fdm,
        AG_force_model,
        Energy_model,
        hparams,
        soap_params
        )


In [15]:
energy, energy_uncertainty = predictor.predict_energy_single_snapshot_r(traj_train[0].copy())
print(energy, energy_uncertainty)

True
[0.14562514] [0.30684346]


In [16]:
fande_calc = FandeCalc(predictor)

In [38]:
atoms = traj_train[33].copy()

atoms.set_calculator(fande_calc)

atoms.get_potential_energy()

Atomic group force model is not defined. Cannot predict forces. Returning zeros.


array([0.14562514], dtype=float32)

In [39]:
fande_calc.save_predictor("prrrrr.pt")

Saving predictor requires humongous amount of memory! Spare some dozens of GBs!


In [3]:
import torch
from fande.ase import FandeCalc

predictor_loaded = torch.load("prrrrr.pt")

fande.models module imported...


In [4]:
fande_calc_loaded = FandeCalc(predictor_loaded)

In [6]:
from ase import io
atoms = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="10")


atoms.set_calculator(fande_calc_loaded)

In [7]:
atoms.get_potential_energy()

Atomic group force model is not defined. Cannot predict forces. Returning zeros.


/home/dlbox2/repos/fande/fande/data/data_module.py:435: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/torch/csrc/utils/tensor_new.cpp:245.)
  X = torch.tensor(X_np_batched,dtype=torch.float32)


array([0.14562514], dtype=float32)

In [8]:
atoms.get_forces()

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])